## 1. ПРЕДВАРИТЕЛЬНАЯ НАСТРОЙКА ДАННЫХ

### 1.1 Настройка библиотек

#### 1.1.1 Библиотеки для работы с директориями

In [ ]:
import os
import sys
import csv

#### 1.1.2 Библиотеки для работы с данными

In [ ]:
# Для обработки таблиц и работы с массивами
import re
import numpy as np
import pandas as pd
import openpyxl
import json

import warnings

# Отключение предупреждений, возникающих при чтении данных
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl.styles.stylesheet')

#### 1.1.3 Библиотеки для работы с парсингом

In [ ]:
# Библиотеки для парсинга (без открытия браузера)
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests

# Библиотеки для парсинга (с открытием браузера)
from selenium import webdriver
from selenium.webdriver.chrome.options import Options                         # <- для настройки параметров браузера 
from selenium.webdriver.chrome.service import Service as ChromeService        # <- для настройки параметров браузера 
from selenium.webdriver.common.by import By                                   # <- для поиска элемента по условию (кнопки прогрузки)
from selenium.webdriver.support.ui import WebDriverWait                       # <- таймер для браузера
from selenium.webdriver.support import expected_conditions as EC              # <- для ожидания определенных условий на странице
from selenium.common.exceptions import TimeoutException, WebDriverException   # <- чтобы избежать ошибки в случае истечения времени

# Библиотека для автоматического обновления/скачивания/удаления драйвера
from webdriver_manager.chrome import ChromeDriverManager

#### 1.1.4 Прочие библиотеки

In [ ]:
from time import sleep

#### 1.1.5 Импорт кода из папки Source

In [ ]:
import importlib.util

module_name = 'parameters_erz_parsing'
module_path = os.path.abspath('../source/parameters_erz_parsing.py')

spec = importlib.util.spec_from_file_location(module_name, module_path)
parameters_erz_parsing = importlib.util.module_from_spec(spec)
spec.loader.exec_module(parameters_erz_parsing)

from source.parameters_erz_parsing import *

### 1.2 Настройка директорий

In [ ]:
this_directory = os.getcwd()

dipl_directory = this_directory.split('main_folder')[0]
data_directory = dipl_directory + 'raw_data'

# В Windows подключение к данным выглядит так:
xlsx_directory = data_directory + '\\excel\\'
dcsv_directory = data_directory + '\\csv\\erz\\'
json_directory = data_directory + '\\json\\erz\\'

# Директория выгруженных и первично обработанных данных
data_clean_directory = this_directory.split('notebooks')[0] + 'data'

### 1.3 Пересохранение данных из excel в csv (если это требуется)
ВАЖНО: помимо просто пересохранения файла ещё вычленяем ссылки на ЖК для парсинга

In [ ]:
# Функция пересохранения файла
def excel_to_csv(name_file_excel: str, page_save: str, name_file_csv: str):
    
    # -----------------------------------------------------------------
    # выбираем файл 
    try:
        wb = openpyxl.load_workbook(name_file_excel, data_only=True)
    except Exception as e:
        print(f'Ошибка при загрузке файла .xlsx: {e}')
        sys.exit(1)
    
    # -----------------------------------------------------------------
    # выгружаем лист
    try:
        ws = wb[page_save]
    except KeyError:
        print(f"Лист с именем '{page_save}' не найден в файле.")
        sys.exit(1)
    
    # -----------------------------------------------------------------
    # из столбца с гиперссылками вытаскиваем ссылки на ЖК
    column_letter = 5
    hyperlinks = []
    
    for row in ws.iter_rows(min_row=2, min_col=column_letter, max_col=column_letter):
        cell = row[0]
        cell_value = cell.value if cell.value else ""
        hyperlink = cell.hyperlink.target if cell.hyperlink else ""
        hyperlinks.append((cell_value, hyperlink))
    
    # -----------------------------------------------------------------
    # пересохраняем файл
    file_open = pd.read_excel(name_file_excel, sheet_name=page_save, engine='openpyxl')
    
    # Добавляем новые столбцы
    file_open['Наименование ЖК'] = [item[0] for item in hyperlinks]
    file_open['Ссылка на ЖК'] = [item[1] for item in hyperlinks]
    
    file_open.to_csv(name_file_csv, index=False)

In [ ]:
# data = '01012025'

In [ ]:
# excel_to_csv(xlsx_directory + 'top_gk_' + data + '_M.xlsx', 'Топ ЖК', dcsv_directory + 'top_gk_M.csv')
# excel_to_csv(xlsx_directory + 'top_gk_' + data + '_MO.xlsx', 'Топ ЖК', dcsv_directory + 'top_gk_MO.csv')
# excel_to_csv(xlsx_directory + 'top_gk_' + data + '_LO.xlsx', 'Топ ЖК', dcsv_directory + 'top_gk_LO.csv')
# excel_to_csv(xlsx_directory + 'top_gk_' + data + '_SPB.xlsx', 'Топ ЖК', dcsv_directory + 'top_gk_SPB.csv')

### 1.4 Обработка таблиц с топом ЖК

#### 1.4.1 Первичная обработка списка ЖК

In [ ]:
# Функция очистки и преобразования таблицы для получения ссылок
def update_table(df: str, to_save = True):

    # -----------------------------------------------------------------
    # <-- попытка прочитать файл
    try:
        pd_region = pd.read_csv(df)
    except:
        return None
    
    # -----------------------------------------------------------------
    # <-- попытка запустить преобразвания наименований Застройщика
    try:
        pd_region['Застройщик'] = pd_region['Застройщик'].str.upper().str.replace('ГК ', '', regex=False)
    except:
        pass
    
    # -----------------------------------------------------------------
    # <-- попыка избавиться от лишних столбцов
    try:
        pd_region = pd_region.drop(columns=['№', '+/-'])
    except:
        pass
    
    # -----------------------------------------------------------------
    # <-- сохранение файла, если то необходимо
    if to_save:
        pd_region.to_csv(df, index=False)
    
    # -----------------------------------------------------------------
    # <-- возвращаем таблицу
    return pd_region

#### 1.4.2 Выгрузка таблиц

In [ ]:
os.chdir(dcsv_directory)

In [ ]:
csv_msk = update_table('top_gk_M.csv')
csv_mo = update_table('top_gk_MO.csv')
csv_spb = update_table('top_gk_SPB.csv')
csv_lo = update_table('top_gk_LO.csv')

#### 1.4.3 Репрезентация таблиц

In [ ]:
csv_msk

In [ ]:
csv_mo

In [ ]:
csv_spb

In [ ]:
csv_lo

---
## 2. ПАРСИНГ ДАННЫХ

### 2.1 Класс парсинга с использованием Selenium

In [ ]:

'''
===============================================================================
ПАРСИНГ
===============================================================================
'''
class ParsingSelenium():
    
    # ------------------------------------------------------------------------
    ''' ПРЕДВАРИТЕЛЬНАЯ ПОДГОТОВКА '''
    def __init__(self, headless = True):
        
        # Инициируем настройки для хрома
        chrome_options = Options()
        
        if headless:
            chrome_options.add_argument("--headless=new")  # <-- Фоновый режим
            chrome_options.add_argument("--disable-gpu")   # <-- Отключение GPU (для headless)
            chrome_options.add_argument("--no-sandbox")    # <-- Отключение sandbox (для некоторых систем)
        
        service = ChromeService(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service = service, options = chrome_options)
    
    
    # ------------------------------------------------------------------------
    ''' ФУНКЦИЯ ОТКРЫТИЯ ВЕБ-СТРАНИЦЫ ДЛЯ ПАРСИНГА '''
    def _link_get(self, link: str) -> object:
        return self.driver.get(link)
    
        
    # ------------------------------------------------------------------------
    ''' ФУНКЦИЯ ОТКРЫТИЯ СТРАНИЦЫ '''
    def _process_retry(self, func, *args, retries = 3, **kwargs):
        for attempt in range(1, retries + 1):
            try:
                return func(*args, **kwargs)
            
            except (TimeoutException, WebDriverException) as e:
                print(f'Ошибка при загрузке данных: {e}')
                print(f'Попытка {attempt} из {retries}...')
                
                if attempt < retries:
                    sleep(2 ** attempt) # <-- Экспоненциальная задержка между попытками
                else:
                    raise               # <-- Поднимаем исключение, если все попытки неудачны


### 2.2 Класс парсинга параметров ЖК

In [ ]:
'''
===============================================================================
ПОИСК ИНФОРМАЦИИ ПО ЖК
===============================================================================
'''
class ParsingERZ_ZHK():
    
    # ------------------------------------------------------------------------
    ''' ПРЕДВАРИТЕЛЬНАЯ ПОДГОТОВКА '''
    def _init_(self, tbl):
        self.tbl = tbl
        self.col_names = tbl['Наименование ЖК']
        self.col_links = tbl['Ссылка на ЖК']
        
        self.dct_names = {}
        for i in range(len(self.col_names)):
            self.dct_names[self.col_names[i]] = self.col_links[i]
            
            
    # ------------------------------------------------------------------------
    ''' ФУНКЦИЯ ЗАПУСКА ПАРСИНГА '''
    def _parsingZHK(self, objects: dict, timer = 10):
        
        # 1. Создание класса парсинга
        SelPars = ParsingSelenium()
        
        # 2. Создание словаря с параметрами 
        parametersZHK = {}
        
        # 3. Цикл для каждого проекта
        for zhk in objects:

            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            print(zhk)
            
            # --> добавление наименования ЖК в словарь
            parametersZHK[zhk] = {}
            
            # --> создание ссылки
            link = objects[zhk]
            # --> ресурс сайта
            mainSoup = SelPars._process_retry(SelPars._link_get, link)
            
            # --> введение параметра временной задержки
            wait = WebDriverWait(SelPars.driver, timer)
            
            # --> пробуем найти параметры ЖК
            try:
                # --> пустые списки, которые необходимо заполнить параметрами, если они есть
                rowTabs = []
                rowVals = []
                tries_count = 0
                
                # --> запуск цикла поиска парамеров
                while (rowTabs == [] or rowVals == []) and tries_count <= timer:
                
                    # --> поиск таблицы 
                    webTableZhk = wait.until(EC.presence_of_element_located((By.XPATH, class_house)))
            
                    # --> поиск всех строк наименований параметров и их значений
                    rowTabs = webTableZhk.find_elements(By.XPATH, class_name)
                    rowVals = webTableZhk.find_elements(By.XPATH, class_values)
                
                    # --> иногда страницы не успевают прогрузиться, нужно подождать и попробовать снова
                    if rowTabs == [] or rowVals == []:
                        tries_count += 1
                        sleep(5)
                        
                    else:
                        tries_count = timer + 1
                    
                # --> возвращение всех параметров            
                for i in range(len(rowTabs)):
                    
                    # параметры
                    parameter_name = rowTabs[i].text
                    parameter_value = rowVals[i].text.replace('\n', ' ')
                    
                    # добавление в общий словарь
                    parametersZHK[zhk][parameter_name] = parameter_value
                    
                    # принт параметра
                    # print(' - ', rowTabs[i].text + ': ' + rowVals[i].text.replace('\n', ' '))
            
            # ---------- ошибка при условии, что не найдено ----------
            except TimeoutException:
                print('Данные не найдены!')
            
            # print()
            
        return parametersZHK
            

### 2.3 Запуск выгрузки для регионов

In [ ]:
# dfs_regions = {
#     'params_msk': csv_msk,
#     'params_mo': csv_mo,
#     'params_spb': csv_spb,
#     'params_lo': csv_lo
#     }

# for region in dfs_regions:
#     erz_zhk = ParsingERZ_ZHK(dfs_regions[region])
#     erzZHKparams = erz_zhk._parsingZHK(erz_zhk.dct_names)  

#     with open(json_directory + region + '.json', 'w', encoding='utf-8') as file:
#         json.dump(erzZHKparams, file, ensure_ascii=False, indent=4) 

#     print('\n==================================================')
#     print("Data has been saved to '{}.json'".format(json_directory + region))
#     print('==================================================\n')

---
## 3. ОБРАБОТКА ДАННЫХ

### 3.1 Выгрузка уникальных значений для каждого параметра (столбца)
(необходимо для того, чтобы посмотреть, какие значения во что превращать)

In [ ]:
# Функция универсальных значений
def unique_check(df: object, directory = '', name_to_save = 'unique_data.csv'):

    unique_columns = {}

    for column in df.columns:
        unique_values = df[column].drop_duplicates().dropna().tolist()
        unique_columns[column] = unique_values

    # Найдем максимальную длину среди всех списков уникальных значений
    max_length = max(len(values) for values in unique_columns.values())

    # Создадим новый словарь с выровненными списками
    aligned_columns = {}
    for column, values in unique_columns.items():
        
        # Заполним недостающие места значением NaN
        aligned_values = values + [None]*(max_length - len(values))
        aligned_columns[column] = aligned_values

    # Создадим новый DataFrame
    unique_df = pd.DataFrame(aligned_columns).T

    # Сохранение в новый CSV файл
    unique_df.to_csv(directory + name_to_save)
    print("Уникальные значения сохранены в '{}'".format(directory + name_to_save))
    

In [ ]:
# total_df = pd.DataFrame()

# list_regions = ['params_m.json', 'params_mo.json', 'params_spb.json', 'params_lo.json']
# for name in list_regions:
#     df = pd.read_json(json_directory + name).T.reset_index()
#     df = df.rename(columns={'index':'ЖК'})

#     total_df = pd.concat([total_df, df], ignore_index=True)

# total_df.to_csv(dcsv_directory + 'params_all_regions.csv')
# unique_check(total_df, dcsv_directory, 'total_unique_data.csv')

### 3.2 Преобразование текстовых параметров в числовые ("эмбеддинг")  
Необходимый этап, поскольку данные из ЕРЗ не представляются сразу в измеримых величинах (ниже показано)

#### 3.2.0 Репрезентация уникальных значений

In [ ]:
df_unique = pd.read_csv(dcsv_directory + 'total_unique_data.csv')
df_unique = df_unique.rename(columns={'Unnamed: 0': 'Параметр'})

df_unique.head()

Пояснение: для любого параметра можно заметить, что он состоит из двух элементов:
- Метрика - имеет категориальный/текстовый формат, который описывает значение параметра ("средний"/"малый", "от ... до ..." и проч.);  
- Оценка - имеет числовой формат.  

Необходимо их разделить друг от друга + преобразовать метрику в число.

#### 3.2.1 Обработка данных посредством разделения "метрик" и "оценок"

In [ ]:
'''
===============================================================================
Эмбеддинг
===============================================================================
'''
class EmbeddingByHands():
    
    # ------------------------------------------------------------------------
    ''' ПРЕДВАРИТЕЛЬНАЯ ПОДГОТОВКА '''
    def __init__(self, name_change: str, name_reference: str, directory_change = '', directory_reference = ''):
        self.table_to_change = pd.read_csv(directory_change + name_change, encoding='utf-8')
        self.reference_table = pd.read_csv(directory_reference + name_reference, encoding='utf-8')
        
        
    # ------------------------------------------------------------------------
    ''' ФУНЦИЯ ПРОВЕРКИ НА ТО, ЧТО СТРОКА ЯВЛЯЕТСЯ ЧИСЛОМ '''
    def _is_number(self, string: str) -> bool:
        try:
            float(string.replace(',', '.'))
            return True
        
        except (ValueError, TypeError):
            return False
        
    
    # ------------------------------------------------------------------------
    ''' ФУНКЦИЯ НОРМАЛИЗАЦИИ ТЕКСТА ДЛЯ ПОВЫШЕНИЯ НАДЁЖНОСТИ СРАВНЕНИЯ '''
    def _normalize_text(self, text: str) -> str:
        if not isinstance(text, str):
            return ''
    
        # Приводим к нижнему регистру и удаляем лишние пробелы
        text = text.lower().strip()
        text = re.sub(r'\s+', ' ', text)
    
        # Заменяем запятые на точки в числах
        text = re.sub(r'(\d),(\d)', r'\1.\2', text)
    
        return text
    
    
    # ------------------------------------------------------------------------
    ''' ФУНКЦИЯ РАЗДЕЛЕНИЯ МЕТРИКИ И ОЦЕНКИ '''
    def _split_metric_rating(self, cell_value: object):

        if not isinstance(cell_value, str) or cell_value.strip() == '':
            return None, None
    
        # Разделяем строку по пробелам
        parts = cell_value.strip().split()
    
        # Проверяем, что последняя часть - это число (оценка)
        if parts and self._is_number(parts[-1]):
            rating = float(parts[-1].replace(',', '.'))
            metric = ' '.join(parts[:-1])
            return metric, rating
    
        return cell_value, None
    
    
    # ------------------------------------------------------------------------
    ''' ФУНКЦИЯ СОЗДАНИЯ СЛОВАРЯ СООТВЕТСТВИЯ МЕТРИК И ЧИСЛОВЫХ ЗНАЧЕНИЙ НА ОСНОВЕ СПРАВОЧНОЙ ТАБЛИЦЫ '''
    def _build_metric_mapping_from_reference(self) -> dict:
        
        try:
            # Загружаем справочную таблицу
            df = self.reference_table
        
            # Создаем словарь для хранения соответствий
            metric_mapping = {}
        
            # Обрабатываем каждую строку таблицы (параметр)
            for row_idx in range(df.shape[0]):
                parameter = df.iloc[row_idx, 0]  # Название параметра
                normalized_parameter = self._normalize_text(parameter)
                
                # Создаем вложенный словарь для этого параметра
                if normalized_parameter not in metric_mapping:
                    metric_mapping[normalized_parameter] = {}
                
                # Обрабатываем каждое возможное значение параметра
                for col_idx in range(1, df.shape[1]):
                    cell_value = df.iloc[row_idx, col_idx]
                    
                    if pd.isna(cell_value) or cell_value == '':
                        continue
                    
                    # Разделяем метрику и оценку
                    metric, rating = self._split_metric_rating(str(cell_value))
                    
                    if metric:
                        # Нормализуем метрику для более надежного сравнения
                        normalized_metric = self._normalize_text(metric)
                        
                        # Запоминаем соответствие (используем позицию в таблице)
                        metric_mapping[normalized_parameter][normalized_metric] = col_idx
            
            return metric_mapping
    
        except Exception as e:
            print(f'Ошибка при создании словаря соответствий: {e}')
            return {}


    # ------------------------------------------------------------------------
    ''' ФУНКЦИЯ ПРЕОБРАЗОВАНИЯ ТЕКСТОВОЙ МЕТРИКИ В ЧИСЛОВОЕ ЗНАЧЕНИЕ С УЧЁТОМ ПАРАМЕТРА И СПРАВОЧНОЙ ТАБЛИЦЫ '''
    def _convert_metric_to_numeric(self, metric, parameter=None, metric_mapping=None):
        
        if not isinstance(metric, str) or metric.strip() == '':
            return None
    
        original_metric = metric.strip()
        normalized_metric = self._normalize_text(original_metric)
        
        # Если есть словарь соответствий и известен параметр, пытаемся использовать его
        if metric_mapping and parameter:
            normalized_parameter = self._normalize_text(parameter)
            
            if normalized_parameter in metric_mapping and normalized_metric in metric_mapping[normalized_parameter]:
                return metric_mapping[normalized_parameter][normalized_metric]
    
        # Если не удалось найти соответствие в словаре, используем общую логику
        # Качественные оценки
        if normalized_metric in ["низкий", "плохой", "минимальный"]:
            return 1
        elif normalized_metric == "средний":
            return 2
        elif normalized_metric in ["высокий", "хороший", "максимальный"]:
            return 3
        elif normalized_metric in ["нет", "отсутствует", "нет подземного паркинга", "нет лифта"]:
            return 0
        elif normalized_metric in ["есть", "да", "имеется"]:
            return 1
        
        # Диапазоны значений ("от ... до")
        range_match = re.search(r'от\s+(\d+[.,]?\d*)\s+до\s+(\d+[.,]?\d*)', normalized_metric)
        if range_match:
            lower = float(range_match.group(1).replace(',', '.'))
            upper = float(range_match.group(2).replace(',', '.'))
            return (lower + upper) / 2
        
        # "менее X"
        less_match = re.search(r'менее\s+(\d+[.,]?\d*)', normalized_metric)
        if less_match:
            value = float(less_match.group(1).replace(',', '.'))
            return value / 2
        
        # "более X"
        more_match = re.search(r'более\s+(\d+[.,]?\d*)', normalized_metric)
        if more_match:
            value = float(more_match.group(1).replace(',', '.'))
            return value * 1.5
    
        # Числовые значения с единицами измерения
        num_match = re.search(r'(\d+[.,]?\d*)\s*(?:м|метров|км)?', normalized_metric)
        if num_match:
            return float(num_match.group(1).replace(',', '.'))
    
    
    # ------------------------------------------------------------------------
    ''' ФУНКЦИЯ ОБРАБОТКИ CSV-ФАЙЛА И СОЗДАНИЕ НОВОГО DATAFRAME С РАЗДЕЛЁННЫМИ МЕТРИКАМИ И ОЦЕНКАМИ '''
    def _process_csv(self):
        
        try:
            # Создаем словарь соответствий на основе справочной таблицы
            metric_mapping = self._build_metric_mapping_from_reference()
            
            # Загружаем данные
            df = self.table_to_change
                        
            # Создаем новый DataFrame для результатов
            result_df = pd.DataFrame()
        
            # Копируем названия ЖК в результирующий DataFrame
            if df.shape[0] > 0:
                result_df['ЖК'] = df.iloc[:, 0]
        
            # Обрабатываем каждый столбец данных (параметр)
            for col_idx, col_name in enumerate(df.columns[1:], 1):
                # Временные списки для метрик, числовых метрик и оценок
                metrics = []
                numeric_metrics = []
                ratings = []
            
                # Обрабатываем каждую ячейку в столбце
                for row_idx in range(df.shape[0]):
                    cell_value = df.iloc[row_idx, col_idx]
                    metric, rating = self._split_metric_rating(str(cell_value))
                    numeric_metric = self._convert_metric_to_numeric(metric, col_name, metric_mapping)
                
                    metrics.append(metric)
                    numeric_metrics.append(numeric_metric)
                    ratings.append(rating)
            
                # Добавляем результаты в итоговый DataFrame
                result_df[f'{col_name}_метрика'] = metrics
                result_df[f'{col_name}_числовая_метрика'] = numeric_metrics
                result_df[f'{col_name}_оценка'] = ratings
        
            return result_df
        
        except Exception as e:
            print(f'Ошибка при обработке файла: {e}')
            return pd.DataFrame()
        
    
    # ------------------------------------------------------------------------
    ''' ФУНКЦИЯ ОБРАБОТКИ CSV-ФАЙЛА И СОЗДАНИЕ НОВОГО DATAFRAME В ДЛИННОМ ФОРМАТЕ '''
    def _process_csv_long_format(self):
        
        try:
            # Создаем словарь соответствий на основе справочной таблицы
            metric_mapping = self._build_metric_mapping_from_reference()
        
            # Загружаем данные
            df = self.table_to_change
        
            # Создаем списки для длинного формата
            rows = []
        
            # Обрабатываем каждую строку (ЖК)
            for row_idx in range(df.shape[0]):
                zhk_name = df.iloc[row_idx, 0]
            
                # Обрабатываем каждый параметр для данного ЖК
                for col_idx, param_name in enumerate(df.columns[1:], 1):
                    cell_value = df.iloc[row_idx, col_idx]
                    metric, rating = self._split_metric_rating(str(cell_value))
                    numeric_metric = self._convert_metric_to_numeric(metric, param_name, metric_mapping)
                
                    # Добавляем строку в результат
                    rows.append({
                        'ЖК': zhk_name,
                        'Параметр': param_name,
                        'Метрика': metric,
                        'Числовая_метрика': numeric_metric,
                        'Оценка': rating
                    })
        
            return pd.DataFrame(rows)
    
        except Exception as e:
            print(f'Ошибка при обработке файла: {e}')
            return pd.DataFrame()

Запуск класса обработки

In [ ]:
embed = EmbeddingByHands('params_all_regions.csv', 'total_unique_data.csv', directory_change=dcsv_directory, directory_reference=dcsv_directory)

In [ ]:
# Обработка в широком формате
result_wide_df = embed._process_csv()
    
# Обработка в длинном формате
# result_long_df = embed._process_csv_long_format()

#### 3.2.2 Дополнительные корректировки перед сохранением

In [ ]:
wide_total = result_wide_df.drop(columns=['ЖК', 'ЖК_числовая_метрика', 'ЖК_оценка'])
wide_total = wide_total.rename(columns={'ЖК_метрика': 'ЖК'})

wide_total

In [ ]:
cols_metrics_numb = ['ЖК']
cols_values = ['ЖК']

cols_metrics_rename = {}
cols_values_rename = {}

i_met = 0
i_est = 0

# Цикл отбора параметров для двух типов таблиц
for col in wide_total.columns:
    if col.endswith('числовая_метрика'):
        cols_metrics_numb.append(col)
        
        i_met += 1
        cols_metrics_rename[col] = 'X_{}'.format(i_met)
        
    elif col.endswith('оценка'):
        cols_values.append(col)
        
        i_est += 1
        cols_values_rename[col] = 'X_{}'.format(i_est)
        
# Создание новых таблиц данных, переименовывание и их сохранение
X_metrics = wide_total[cols_metrics_numb]
X_metrics = X_metrics.rename(columns=cols_metrics_rename)

X_values = wide_total[cols_values]
X_values = X_values.rename(columns=cols_values_rename)

In [ ]:
X_metrics

In [ ]:
X_values

In [ ]:
# Сохранение таблиц
X_metrics.to_csv(data_clean_directory + 'ERZ_X_metrics.csv', index=False, encoding='utf-8')
X_values.to_csv(data_clean_directory + 'ERZ_X_values.csv', index=False, encoding='utf-8')